In [1]:
import pandas as pd
import numpy as np

In [4]:
data = pd.DataFrame('data/data.csv')

ValueError: DataFrame constructor not properly called!

In [ ]:
data.head()

In [ ]:


# Suponhamos que você tenha um DataFrame chamado 'data' com colunas 'Date' e 'Price'
# 'Date' deve conter as datas e 'Price' os preços dos ativos.

# Calculando o retorno diário
data['Return'] = data['Price'].pct_change()

# Define a estratégia de backtest (por exemplo, uma média móvel simples)
short_window = 10
long_window = 50

data['Short_MA'] = data['Price'].rolling(window=short_window).mean()
data['Long_MA'] = data['Price'].rolling(window=long_window).mean()

# Cria um sinal de compra (quando a média móvel curta cruza acima da longa) e um sinal de venda (quando a curta cruza abaixo da longa)
data['Signal'] = 0
data.loc[data['Short_MA'] > data['Long_MA'], 'Signal'] = 1
data.loc[data['Short_MA'] < data['Long_MA'], 'Signal'] = -1

# Calcula os retornos da estratégia
data['Strategy_Return'] = data['Signal'].shift(1) * data['Return']

# Calcula o retorno acumulado
data['Cumulative_Return'] = (1 + data['Strategy_Return']).cumprod()

# Visualiza os resultados
print(data)
